In [1]:
%pip install pyproj
#패키지 설치, import
import pyproj

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\gymdo\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [2]:
import folium
import pandas as pd
import numpy
from folium.plugins import MarkerCluster
#패키지 import

In [3]:
df = pd.read_csv('./data/편의시설/부산광역시_종합병원 현황_20211231.csv'
, encoding='UTF-8') #부산광역시 종합병원 현황 읽어오기 
hos_sub_df = df[df['도로명주소'].str.contains('기장군')]#관리기관명이 기장군인 것만 자름
print(hos_sub_df[['위도','경도','의료기관명']]) #기장군 병원 출력 테스트, 1개 존재


           위도          경도      의료기관명
27  35.321839  129.244334  동남권원자력의학원


In [4]:
df = pd.read_csv('./data/편의시설/부산광역시_노인교실 현황_08_31_2021.csv'
, encoding='cp949') #부산광역시 노인교실 현황 읽어오기 
class_sub_df = df.loc[df['구군명'].isin(['기장군'])]#관리기관명이 기장군인 것만 자름
print(class_sub_df[['위도','경도','노인교실명']]) #기장군 노인교실 출력 테스트, 3개 존재



            위도          경도          노인교실명
161  35.242651  129.217226         기장노인대학
162  35.273559  129.150092         철마노인대학
163  35.245678  129.217469  기장군지회 부설 노인대학


In [21]:
df = pd.read_csv('./data/편의시설/fulldata_01_01_06_P_약국.csv'
, engine='python',encoding='cp949') #전국 약국 현황 읽어오기 
newDF=df.loc[df['소재지전체주소'].str.contains('기장군',na=False)]#기장만 끊어서
proj_UMTK = pyproj.Proj('epsg:5174') #오래된 지리원 표준
proj_WGS84 = pyproj.Proj('epsg:4326') #경위도 좌표계
newPGS = pd.DataFrame()#빈 데이터프레임 생성
for x1, y1, add in zip(newDF['좌표정보(x)'],newDF['좌표정보(y)'],newDF['사업장명']):
    x2, y2 = pyproj.transform(proj_UMTK,proj_WGS84,y1,x1) #좌표 변환
    dataToInsert = pd.DataFrame({'위도':[x2],'경도':[y2],'사업장명':[add]})#넣을 데이터
    newPGS = pd.concat([newPGS,dataToInsert],ignore_index=True)#newPGS에 데이터 넣기
newPGS = newPGS.replace([numpy.inf],numpy.nan)#inf nan으로 변환
newPGS=newPGS.dropna()#nan 제거
print(newPGS.head(10))#출력 테스트


C:\Users\gymdo\AppData\Local\Temp\ipykernel_36500\3320736179.py:8: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  x2, y2 = pyproj.transform(proj_UMTK,proj_WGS84,y1,x1) #좌표 변환


           위도          경도      사업장명
0   35.239615  129.170771      고촌약국
1   35.334489  129.165826      누리약국
2   35.319692  129.177448      일신약국
5   35.243909  129.216481      건강약국
6   35.245290  129.214733      미래약국
7   35.224872  129.227949      대변약국
8   35.324108  129.175976      제일약국
9   35.324508  129.180395   우리온누리약국
10  35.328929  129.172001      햇살약국
11  35.320717  129.242051  원자력 가람약국


In [19]:
kizang_pos= [35.23876594878238, 129.21575390899562] ## 기장읍 사무소
m = folium.Map(location=kizang_pos, zoom_start=17,width=750, 
               height=500)#지도 17의 줌으로 생성               
coords = hos_sub_df[['위도','경도','의료기관명']]#자료에서 위도, 경도, 경로당명 가져와서
coords2 = newPGS[['위도','경도','사업장명']]
coords3 = class_sub_df[['위도','경도','노인교실명']]
pgs_marker_cluster = MarkerCluster().add_to(m)
hos_marker_cluster = MarkerCluster().add_to(m)#MarkerCluster을 지도에 추가하고
class_marker_cluster = MarkerCluster().add_to(m)
for lat,long,tool in zip(coords['위도'],coords['경도'],coords['의료기관명']):#lat, long, tool을 이터레이터로 만들어서
    folium.Marker([lat,long], icon=folium.Icon(color='green'),tooltip=tool).add_to(hos_marker_cluster)#초록색 마커로 마우스 갖다대면 이름 보이게 표시
for lat, long,tool in zip(coords2['위도'],coords2['경도'],coords2['사업장명']):
    folium.Marker([lat,long],popup='약국입니다',icon = folium.Icon(color='red',icon='star'),tooltip=tool).add_to(pgs_marker_cluster)
for lat,long,tool in zip(coords3['위도'],coords3['경도'],coords3['노인교실명']):
    folium.Marker([lat,long],popup='노인교실입니다',icon=folium.Icon(color='blue',icon='star'),tooltip=tool).add_to(class_marker_cluster)
m
